In [ ]:
import os
os.environ['JULIA_PROJECT']="/home/jpd47/proj/HAL/Project.toml"

In [ ]:

import ase
from ase import Atoms
from sklearn.linear_model import BayesianRidge, ARDRegression
#from ase.calculators.castep import Castep
from ase.io import read, write
import quippy
from quippy.potential import Potential
from pathlib import Path
from ACEHAL.HAL import HAL

In [ ]:


fit_configs = read("/home/jpd47/proj/HAL/0_conv.xyz", ":")

calculator = Potential(param_filename="/home/jpd47/proj/HAL/gap_pot/n_max-8__l_max-4__radial_basis-EQUISPACED_GAUSS__n_sparse-1600.xml")

E0s = { "Fe" : -3.09707575, "Ni" : -0.52326479, "Cr" : -5.4579557 }
data_keys = { "E" : "energy", "F" : "forces", "V" : "virial", "Fmax" : 20.0 }
weights = { "E" : 1.0, "F" : 1.0 ,"V": 1.0 }
#weights = { "E_per_atom": 1.0 / 0.001, "F": 1.0 / 0.1, "V_per_atom": 1.0 / 0.01 }

solver = ARDRegression(threshold_lambda=10000, fit_intercept=True, compute_score=True)
#solver = BayesianRidge(fit_intercept=True, compute_score=True)

fit_kwargs={"E0s": E0s, "data_keys": data_keys, "weights": weights}

fixed_basis_info = {"elements": list(E0s.keys()), "smoothness_prior" : ["algebraic", 2] }
optimize_params = {"cor_order": ("int", (2, 3)), "maxdeg": ("int", (3, 10)), "r_cut": ("float", (5.5, 8.0))}

HAL(fit_configs, fit_configs, None, solver,
            fit_kwargs={"E0s": E0s, "data_keys": data_keys, "weights": weights},
            n_iters=100, traj_len=2000, tol=0.2, tol_eps=0.2, tau_rel=0.2,
            ref_calc=calculator, dt=1.0, T_K=600, T_tau=100, P_GPa=1.0,
            basis_optim_kwargs={"n_trials": 20,
                                "timeout" : 10000,
                                "max_basis_len": 100000,
                                "fixed_basis_info": fixed_basis_info,
                                "optimize_params": optimize_params},
            basis_optim_interval=1, file_root="test_HAL",
            test_fraction=0.1)